# 공공날씨 - 동네예보조회

In [1]:
# -*- coding: utf-8 -*-
import datetime as dt
import requests as req
from bs4 import BeautifulSoup
import json 
import pandas as pd 

In [2]:
w_items = {"POP":["강수확률", "%"], 
           "PTY":["강수형태", ""], 
           "R06":["6시간 강수량", "mm"], 
           "REH":["습도", "%"], 
           "S06":["6시간 신적설", "cm"], 
           "SKY":["하늘상태", ""], 
           "T3H":["3시간 기온", "°C"], 
           "TMN":["아침 최저 기온", "°C"], 
           "TMX":["낮 최고 기온", "°C"], 
           "UUU":["풍속(동서성분)","m/s"], 
           "VVV":["풍속(남북성분)","m/s"], 
           "WAV":["파고", "m"], 
           "VEC":["풍향", ""], 
           "WSD":["풍속", "m/s"]}
wind_dict = {0:"N",1:"NNE",2:"NE",3:"ENE",4:"E",5:"ESE",6:"SE",7:"SSE",8:"S",9:"SSW",10:"SW",11:"WSW",12:"W",13:"WNW",14:"NW",15:"NNW",16:"N"}
rain_dict = {0:"없음", 1:"비", 2:"진눈깨비", 3:"눈"}
 
# 가장 최근의 기상청 예보 시간(02:10 이후 3시간 단위) get
def get_rpt_time():
    base_time = dt.datetime.now() - dt.timedelta(hours=2) - dt.timedelta(minutes=10)
    hour = int(base_time.strftime("%H"))
    rehour = str(hour//3*3 + 2).zfill(2)
    return [base_time.strftime("%Y%m%d"), rehour + "00"]
 
# 풍향 계산용
def get_wind_type(i):
    return wind_dict[(i + 22.5 * 0.5)//22.5]
 
def get_sky_status(i):
    if i >= 0 and i <= 2: return "맑음"
    elif i >= 3 and i <= 5: return "구름조금"
    elif i >= 6 and i <= 8: return "구름많음"
    elif i >= 9 and i <= 10: return "흐림"
    
def get_real_value(c, s):
    if c.string == "VEC":
        return get_wind_type(int(s.string))
    elif c.string == "SKY":
        return get_sky_status(int(s.string))
    elif c.string == "PTY":
        return rain_dict[int(s.string)]
    return s.string
 


In [3]:
# have to change with your service_key
service_key = "gC8RUN3kxgc5v1ZNPvEYW%2FspE4YwHYOn7VWWlIQGfGw2fNVEBCrzvE8cHUFEZpxk6jHmkvwHK9RuL2EjzNz4WQ%3D%3D"
nx = 59    # 서울 기준
ny = 127   # 서울 기준
 
date = get_rpt_time()
 
base_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst?ServiceKey={}&base_date={}&base_time={}&nx={}&ny={}"
url = base_url.format(service_key, date[0], date[1], nx, ny)
 
contents = req.get(url)

weather = BeautifulSoup(contents.content, 'lxml-xml')
weather

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL_SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><baseDate>20210416</baseDate><baseTime>0800</baseTime><category>POP</category><fcstDate>20210416</fcstDate><fcstTime>1200</fcstTime><fcstValue>60</fcstValue><nx>59</nx><ny>127</ny></item><item><baseDate>20210416</baseDate><baseTime>0800</baseTime><category>PTY</category><fcstDate>20210416</fcstDate><fcstTime>1200</fcstTime><fcstValue>1</fcstValue><nx>59</nx><ny>127</ny></item><item><baseDate>20210416</baseDate><baseTime>0800</baseTime><category>R06</category><fcstDate>20210416</fcstDate><fcstTime>1200</fcstTime><fcstValue>3</fcstValue><nx>59</nx><ny>127</ny></item><item><baseDate>20210416</baseDate><baseTime>0800</baseTime><category>REH</category><fcstDate>20210416</fcstDate><fcstTime>1200</fcstTime><fcstValue>80</fcstValue><nx>59</nx><ny>127</ny></item><item><baseDate>20210416</baseDate><baseTime>0800</baseTime><ca

In [4]:
print("\t **************************************")
print("\t Report Time :", date[0], date[1])
print("\t **************************************")
for item in weather.find_all("item"):
    print("\t", w_items[item.category.string][0], " : ", get_real_value(item.category, item.fcstValue), end = "")
    if w_items[item.category.string][1] != "":
        print(w_items[item.category.string][1])
    else:
        print()


	 **************************************
	 Report Time : 20210416 0800
	 **************************************
	 강수확률  :  60%
	 강수형태  :  비
	 6시간 강수량  :  3mm
	 습도  :  80%
	 6시간 신적설  :  0cm
	 하늘상태  :  구름조금
	 3시간 기온  :  12°C
	 풍속(동서성분)  :  3.1m/s
	 풍향  :  SW
	 풍속(남북성분)  :  2.8m/s
